In [2]:
import fitz  
from PIL import Image
import pytesseract
import io
import re
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [16]:
from Jiahao.sort_toc_line import sort_toc_lines
from Jiahao.index_finder import extract_toc_pages
from Jiahao.toc_filter import is_valid_toc_line
from Jiahao.sort_toc_line import separate_on_number

ImportError: cannot import name 'separate_on_number' from 'Jiahao.sort_toc_line' (c:\Users\j7843\Documents\DAIA\GEA_llm_pdf_processing_group17\Jiahao\sort_toc_line.py)

In [15]:
def extract_toc_pages(pdf_path, lang='deu'):
    doc = fitz.open(pdf_path)  # Open PDF file
    toc_pages = {}

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        pix = page.get_pixmap(dpi=300)
        img = Image.open(io.BytesIO(pix.tobytes()))
        text = pytesseract.image_to_string(img, lang=lang)

        if "inhaltsverzeichnis" in text.lower():
            #toc_pages[page_num + 1] = text  # store text with human-friendly page number
            return text
        
text = extract_toc_pages("ExtrahiertVig05.pdf")


KeyboardInterrupt: 

In [ ]:
print(text)


#index_line = sort_toc_lines(index_name)
# print(index_name)
# print(index_line)

Anwenderhandbuch GRA3SSü lLKAB
SEP Refigertion Eguipment FON

Schraubenverdichteraggregat Aefgeration Equipment
mit horizontalem Ölabscheider und dem Verdichter F

Inhaltsverzeichnis

1. BESCHREIBUNG 3.1. Allgemeines

3.2. Wartung der Hauptbauteile

1.1. Konstruktiver Aufbau

3.3. Wartungsübersicht
1.2. Arbeitsweise

3.4. Ölwechsel
1.3. Leistungsverstellung

ANLAGEN MIT SACHNUMMER
1.4. Verstellung des inneren 629537. :19321
63 Volumenverhältnisses Vi “

1.5. Überwachungseinrichtungen Kälteschema Anlage 1
1.6. Sicherheitseinrichtungen Übersichtszeichnung SVA FA Anlage 2

1.7. Aggregataufstellung Übersichtszeichnung SVA FH Anlage 3

2. BEDIENUNGSHINWEISE

2.1. Grundsätzliches

& 2.2. Erste Inbetriebnahme
2.3. Betrieb des Verdichteraggregates
2.4. Normale Inbetriebsetzung
2.5. Regelung der Verdichterleistung
2.6. Normale Außerbetriebsetzung

2.7. Außerbetriebsetzung für einen längeren
Zeitraum

3. WARTUNGSANWEISUNG

AFD Spänich 629537.DOC
(Ausgabe/ edition 20.02.95) Revision 0

KÜHLAUTOMA

In [47]:
def is_valid_toc_line(line: str) -> bool:
    line = line.strip()

    # 1. Must start with a number pattern like 1. or 1.1. or 2.3.5.
    if not re.match(r'^\d+(\.\d+)*\.\s?', line):
        return False

    # 2. Must contain alphabetic characters
    if not re.search(r'[A-Za-zÄÖÜäöüß]', line):
        return False

    # 3. Exclude known keywords unrelated to TOC
    if re.search(r'\b(Revision|Ausgabe|edition|Version)\b', line, re.IGNORECASE):
        return False

    # 4. Too short or mostly numeric
    if len(line) < 5:
        return False
    if re.fullmatch(r'\d+(\.\d+)*', line):
        return False
    if re.fullmatch(r'\d{2}\.\d{2}\.\d{2,4}', line):
        return False

    return True

In [ ]:
lines = [line.strip() for line in text.splitlines() if line.strip()]
print(lines)
valid_lines = [line for line in lines if is_valid_toc_line(line)]
print(valid_lines)




process_text = separate_on_number(text)

['Anwenderhandbuch GRA3SSü lLKAB', 'SEP Refigertion Eguipment FON', 'Schraubenverdichteraggregat Aefgeration Equipment', 'mit horizontalem Ölabscheider und dem Verdichter F', 'Inhaltsverzeichnis', '1. BESCHREIBUNG 3.1. Allgemeines', '3.2. Wartung der Hauptbauteile', '1.1. Konstruktiver Aufbau', '3.3. Wartungsübersicht', '1.2. Arbeitsweise', '3.4. Ölwechsel', '1.3. Leistungsverstellung', 'ANLAGEN MIT SACHNUMMER', '1.4. Verstellung des inneren 629537. :19321', '63 Volumenverhältnisses Vi “', '1.5. Überwachungseinrichtungen Kälteschema Anlage 1', '1.6. Sicherheitseinrichtungen Übersichtszeichnung SVA FA Anlage 2', '1.7. Aggregataufstellung Übersichtszeichnung SVA FH Anlage 3', '2. BEDIENUNGSHINWEISE', '2.1. Grundsätzliches', '& 2.2. Erste Inbetriebnahme', '2.3. Betrieb des Verdichteraggregates', '2.4. Normale Inbetriebsetzung', '2.5. Regelung der Verdichterleistung', '2.6. Normale Außerbetriebsetzung', '2.7. Außerbetriebsetzung für einen längeren', 'Zeitraum', '3. WARTUNGSANWEISUNG', 'AFD

ImportError: cannot import name 'separate_on_number' from 'Jiahao.process_text' (c:\Users\j7843\Documents\DAIA\GEA_llm_pdf_processing_group17\Jiahao\process_text.py)

01: Anwenderhandbuch GRA3SSü lLKAB
02: SEP Refigertion Eguipment FON
03: 
04: Schraubenverdichteraggregat Aefgeration Equipment
05: mit horizontalem Ölabscheider und dem Verdichter F
06: 
07: Inhaltsverzeichnis
08: 
09: 1. BESCHREIBUNG
10: 3.1. Allgemeines
11: 
12: 3.2. Wartung der Hauptbauteile
13: 
14: 1.1. Konstruktiver Aufbau
15: 
16: 3.3. Wartungsübersicht
17: 1.2. Arbeitsweise
18: 
19: 3.4. Ölwechsel
20: 1.3. Leistungsverstellung
21: 
22: ANLAGEN MIT SACHNUMMER
23: 1.4. Verstellung des inneren
24: 629537. :19321
25: 63 Volumenverhältnisses Vi “
26: 
27: 1.5. Überwachungseinrichtungen Kälteschema Anlage
28: 1
29: 1.6. Sicherheitseinrichtungen Übersichtszeichnung SVA FA Anlage
30: 2
31: 
32: 1.7. Aggregataufstellung Übersichtszeichnung SVA FH Anlage
33: 3
34: 
35: 2. BEDIENUNGSHINWEISE
36: 
37: 2.1. Grundsätzliches
38: 
39: &
40: 2.2. Erste Inbetriebnahme
41: 2.3. Betrieb des Verdichteraggregates
42: 2.4. Normale Inbetriebsetzung
43: 2.5. Regelung der Verdichterleistung
44: 2.6. No

In [ ]:
sorted_lines = sort_toc_lines(valid_lines)


print(index_name)






if sorted_lines:
    toc_pages[page_num + 1] = sorted_lines  # +1 for human-friendly page number


for page, entries in toc_pages.items():
    print(f"\n--- TOC found on page {page} ---")
    for entry in entries:
        print(entry)

--- Page 12 contains 'Inhaltsverzeichnis' ---
Anwenderhandbuch GRA3SSü lLKAB
SEP Refigertion Eguipment FON

Schraubenverdichteraggregat Aefgeration Equipment
mit horizontalem Ölabscheider und dem Verdichter F

Inhaltsverzeichnis

1. BESCHREIBUNG 3.1. Allgemeines

3.2. Wartung der Hauptbauteile

1.1. Konstruktiver Aufbau

3.3. Wartungsübersicht
1.2. Arbeitsweise

3.4. Ölwechsel
1.3. Leistungsverstellung

ANLAGEN MIT SACHNUMMER
1.4. Verstellung des inneren 629537. :19321
63 Volumenverhältnisses Vi “

1.5. Überwachungseinrichtungen Kälteschema Anlage 1
1.6. Sicherheitseinrichtungen Übersichtszeichnung SVA FA Anlage 2

1.7. Aggregataufstellung Übersichtszeichnung SVA FH Anlage 3

2. BEDIENUNGSHINWEISE

2.1. Grundsätzliches

& 2.2. Erste Inbetriebnahme
2.3. Betrieb des Verdichteraggregates
2.4. Normale Inbetriebsetzung
2.5. Regelung der Verdichterleistung
2.6. Normale Außerbetriebsetzung

2.7. Außerbetriebsetzung für einen längeren
Zeitraum

3. WARTUNGSANWEISUNG

AFD Spänich 629537.DOC
(Aus